In [1]:
import gc

from pprint import pprint

import pandas as pd
import numpy as np
import lightgbm as lgb

from cv import run_cv_model
from utils import print_step, rmse
from cache import get_data, is_in_cache, load_cache, save_in_cache

In [48]:
print('~~~~~~~~~~~~~~~~~~~~~~~~')
print_step('Importing Data')
train, test = get_data()
target = train['deal_probability']
train_ = train['item_id']
train_id = train['item_id']
test_id = test['item_id']
del train, test

print_step('Base LGB')
train_base_lgb, test_base_lgb = load_cache('base_lgb')
train_ = train_base_lgb[['base_lgb']]
test_ = test_base_lgb[['base_lgb']]

print_step('Ridge LGB')
train_ridge_lgb, test_ridge_lgb = load_cache('ridge_lgb')
train_['ridge_lgb'] = train_ridge_lgb['ridge_lgb']
test_['ridge_lgb'] = test_ridge_lgb['ridge_lgb']

print_step('Parent Cat Ridge')
train_pcat_ridge, test_pcat_ridge = load_cache('parent_cat_ridges')
train_pcat_ridge = train_pcat_ridge[[c for c in train_pcat_ridge.columns if 'ridge' in c]]
test_pcat_ridge = test_pcat_ridge[[c for c in test_pcat_ridge.columns if 'ridge' in c]]
train_ = pd.concat([train_, train_pcat_ridge], axis=1)
test_ = pd.concat([test_, test_pcat_ridge], axis=1)

print_step('Parent Region Cat Ridge')
train_rcat_ridge, test_rcat_ridge = load_cache('parent_regioncat_ridges')
train_rcat_ridge = train_rcat_ridge[[c for c in train_rcat_ridge.columns if 'ridge' in c]]
test_rcat_ridge = test_rcat_ridge[[c for c in test_rcat_ridge.columns if 'ridge' in c]]
train_ = pd.concat([train_, train_rcat_ridge], axis=1)
test_ = pd.concat([test_, test_rcat_ridge], axis=1)

print_step('Cat Bin Ridge')
train_catb_ridge, test_catb_ridge = load_cache('cat_bin_ridges')
train_catb_ridge = train_catb_ridge[[c for c in train_catb_ridge.columns if 'ridge' in c]]
test_catb_ridge = test_catb_ridge[[c for c in test_catb_ridge.columns if 'ridge' in c]]
train_ = pd.concat([train_, train_catb_ridge], axis=1)
test_ = pd.concat([test_, test_catb_ridge], axis=1)

print_step('Deep LGB')
train_deep_lgb, test_deep_lgb = load_cache('deep_lgb')
train_['deep_lgb'] = train_deep_lgb['deep_lgb']
test_['deep_lgb'] = test_deep_lgb['deep_lgb']

print_step('Full Text Ridge')
train_full_text_ridge, test_full_text_ridge = load_cache('full_text_ridge')
train_['full_text_ridge'] = train_full_text_ridge['full_text_ridge']
test_['full_text_ridge'] = test_full_text_ridge['full_text_ridge']

print_step('Complete Ridge')
train_complete_ridge, test_complete_ridge = load_cache('complete_ridge')
train_['complete_ridge'] = train_complete_ridge['complete_ridge']
test_['complete_ridge'] = test_complete_ridge['complete_ridge']

print_step('Complete FM')
train_complete_fm, test_complete_fm = load_cache('complete_fm')
train_['complete_fm'] = train_complete_fm['complete_fm']
test_['complete_fm'] = test_complete_fm['complete_fm']

print_step('tffm2')
train_tffm2, test_tffm2 = load_cache('tffm2')
train_['tffm2'] = train_tffm2['tffm2']
test_['tffm2'] = test_tffm2['tffm2']

print_step('tffm3')
train_tffm3, test_tffm3 = load_cache('tffm3')
train_['tffm3'] = train_tffm3['tffm3']
test_['tffm3'] = test_tffm3['tffm3']

print_step('CNN FastText')
train_cnn_ft, test_cnn_ft = load_cache('CNN_FastText')
train_['cnn_ft'] = train_cnn_ft['CNN_FastText']
test_['cnn_ft'] = test_cnn_ft['CNN_FastText']

print_step('CNN FastText 2')
train_cnn_ft, test_cnn_ft = load_cache('CNN_FastText_2')
train_['cnn_ft2'] = train_cnn_ft['CNN_FastText_2']
test_['cnn_ft2'] = test_cnn_ft['CNN_FastText_2']

print_step('CNN FastText 3')
train_cnn_ft, test_cnn_ft = load_cache('CNN_FastText_3')
train_['cnn_ft3'] = train_cnn_ft['CNN_FastText_3']
test_['cnn_ft3'] = test_cnn_ft['CNN_FastText_3']

print_step('CNN FastText 4')
train_cnn_ft, test_cnn_ft = load_cache('CNN_FastText_4')
train_['cnn_ft4'] = train_cnn_ft['CNN_FastText_4']
test_['cnn_ft4'] = test_cnn_ft['CNN_FastText_4']

~~~~~~~~~~~~~~~~~~~~~~~~
[2018-06-17 23:24:15.869012] Importing Data
Train shape: (1503424, 18)
Test shape: (508438, 17)
[2018-06-17 23:24:27.263361] Base LGB
Test shape: (508438, 1)
Train shape: (1503424, 1)
[2018-06-17 23:24:27.549491] Skipped... Loaded cache/train_base_lgb.csv and cache/test_base_lgb.csv from cache!
[2018-06-17 23:24:27.555967] Ridge LGB
Test shape: (508438, 1)
Train shape: (1503424, 1)
[2018-06-17 23:24:27.840575] Skipped... Loaded cache/train_ridge_lgb.csv and cache/test_ridge_lgb.csv from cache!
[2018-06-17 23:24:27.842437] Parent Cat Ridge
Test shape: (508438, 21)
Train shape: (1503424, 22)
[2018-06-17 23:24:40.705401] Skipped... Loaded cache/train_parent_cat_ridges.csv and cache/test_parent_cat_ridges.csv from cache!
[2018-06-17 23:24:40.725418] Parent Region Cat Ridge
Test shape: (508438, 22)
Train shape: (1503424, 23)
[2018-06-17 23:24:54.833154] Skipped... Loaded cache/train_parent_regioncat_ridges.csv and cache/test_parent_regioncat_ridges.csv from cache!
[

In [49]:
from sklearn.linear_model import Lasso

In [64]:
model = Lasso(alpha = 1e-5,positive=True)
model.fit(train_.values, target)

Lasso(alpha=1e-05, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=True, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [63]:
for i in zip(train_.columns, model.coef_):
    print(i)

('base_lgb', 0.0)
('ridge_lgb', 0.6773360055760547)
('cat_title_ridge', 0.0)
('cat_desc_ridge', 0.0)
('cat_desc_char_ridge', 0.0)
('cat_all_text_ridge', 0.0)
('regioncat_title_ridge', 0.0)
('regioncat_desc_ridge', 0.0)
('regioncat_desc_char_ridge', 0.0)
('regioncat_all_text_ridge', 0.0)
('cat_bin_title_ridge', 0.0)
('cat_bin_desc_ridge', 0.0)
('cat_bin_desc_char_ridge', 0.0)
('cat_bin_all_text_ridge', 0.0)
('deep_lgb', 0.14173178052870247)
('full_text_ridge', 0.0)
('complete_ridge', 0.0)
('complete_fm', 0.0)
('tffm2', 0.0)
('tffm3', 0.0)
('cnn_ft', 0.0)
('cnn_ft2', 0.0)
('cnn_ft3', 0.07047802861731235)
('cnn_ft4', 0.15005670729728662)


In [65]:
params = {
    "alpha": 1e-5,
    "positive": True
}
def runLasso(train_X, train_y, val_X, val_y, test_X, params):
    model = Lasso(**params)
    model.fit(train_X, train_y)
    print_step('Predict Val 1/2')
    pred_val_y = model.predict(val_X)
    print_step('Predict Test 2/2')
    pred_test_y = model.predict(test_X)
    return pred_val_y, pred_test_y

In [66]:
results = run_cv_model(train_, test_, target, runLasso, params, rmse, 'lasso_blender')

[2018-06-17 23:41:43.600562] Started lasso_blender fold 1/5
[2018-06-17 23:41:47.445274] Predict Val 1/2
[2018-06-17 23:41:47.450086] Predict Test 2/2
[2018-06-17 23:41:47.469633] lasso_blender cv score 1 : 0.2135611964297842
[2018-06-17 23:41:47.481135] Started lasso_blender fold 2/5
[2018-06-17 23:41:51.456546] Predict Val 1/2
[2018-06-17 23:41:51.461296] Predict Test 2/2
[2018-06-17 23:41:51.480892] lasso_blender cv score 2 : 0.2128683582341426
[2018-06-17 23:41:51.492503] Started lasso_blender fold 3/5
[2018-06-17 23:41:55.467217] Predict Val 1/2
[2018-06-17 23:41:55.472071] Predict Test 2/2
[2018-06-17 23:41:55.491503] lasso_blender cv score 3 : 0.21276185522867955
[2018-06-17 23:41:55.503228] Started lasso_blender fold 4/5
[2018-06-17 23:41:59.577520] Predict Val 1/2
[2018-06-17 23:41:59.582189] Predict Test 2/2
[2018-06-17 23:41:59.601698] lasso_blender cv score 4 : 0.21267414337511575
[2018-06-17 23:41:59.613347] Started lasso_blender fold 5/5
[2018-06-17 23:42:04.218374] Predi

In [ ]:
# alpha 1e-6 Positive Only  # mean cv: 0.21303411626812468
# alpha 1e-5 Positive Only  # mean cv: 0.21303410763895192
# alpha 1e-3 Positive Only  # mean cv: 0.21316230802022598
# alpha 5e-5 Positive Only  # mean cv: 0.21303434967228338
# alpha 5e-6 Positive Only  # mean cv: 0.21303410990268196